<div style="background-image:url(images/bertinoro.jpg); padding: 10px 30px 20px 30px; background-size:cover; background-opacity:50%; border-radius:5px">
<p style="float:right; margin-top:20px; padding: 20px 20px 0px 20px; background:rgba(255,255,255,0.6); border-radius:10px;">
<img width="400px" src=images/obspy_logo_full_524x179px.png?raw=true>
</p>

<h1 style="color:#333">First TIDES Training School</h1>
<h5 style="color:#FFF">Bertinoro (FC), Italy, June 1-5 2015</h5>

<h3 style="color:#EEE">Day 1: Data Acquisition and Processing with ObsPy</h3>

<h2 style="color:#FFF">Process Earthquake Data</h2>
</div>

In this notebook we will use raw data from the datacenters and process it so it is suitable for further processing.

In [ ]:
%pylab inline
from __future__ import print_function
import matplotlib.pylab as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

The pre-downloaded data can be found in `~/Desktop/data/earthquake_data`

In [ ]:
ls -lrt ~/Desktop/data/earthquake_data/waveforms

In [ ]:
ls -lrt ~/Desktop/data/earthquake_data/stations

### Exercise: Process Data

1. Read the raw waveform data for event `GCMT_2015_04_25__Mw_7_9` station for station.
2. Read the corresponing station information.
3. Remove the instrument response. Use the `pre_filt` for filtering the data and filter it between 1 and 200 seconds.
4. Decimate to 10 Hz.
5. Save it all in "processed_data_1_200_seconds"

In [ ]:
import os

data_folder = "/home/tides/Desktop/data/earthquake_data/waveforms/GCMT_2015_04_25__Mw_7_9"
output_folder = "processed_data_1_200_seconds"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

!ls $data_folder

In [ ]:
import obspy

inv = obspy.read_inventory("/home/tides/Desktop/data/earthquake_data/stations/all_stations.xml")